### 데이터 불러오기

In [21]:
import os
import pymupdf4llm
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

pdf_dir = "E:/work/MS_project_2/data/new_pdfs"  # 여러 PDF 파일이 들어 있는 디렉토리


### PDF-> 마크다운 형태로 변환
### source 정보 추가
from langchain.schema import Document

all_docs = []

for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        md_path = pdf_path.replace(".pdf", ".md")

        # 1. PDF → Markdown 저장
        md_text = pymupdf4llm.to_markdown(pdf_path)
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(md_text)

        # 2. Markdown 로딩
        loader = TextLoader(md_path, encoding="utf-8")
        documents = loader.load()

        # 3. ✅ source 메타데이터 추가
        for doc in documents:
            doc.metadata["source"] = filename

        # 4. ✅ 분할 (source 정보가 포함된 문서 기준으로)
        docs_split = text_splitter.split_documents(documents)

        # 5. ✅ all_docs에 청크된 문서 저장
        all_docs.extend(docs_split)


Processing E:/work/MS_project_2/data/new_pdfs\(대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf...
[                                        ] (0/1==[===                                     ] ( 1/1==[======                                  ] ( 2/13==[=========                               ] ( 3/1==[============                            ] ( 4/13==[===============                         ] ( 5/1==[==================                      ] ( 6/13==[=====================                   ] ( 7/1==[========================                ] ( 8/13==[===========================             ] ( 9/1==[==============================          ] (10/13==[=================================       ] (11/1==[====================================    ] (12/13===[========================================] (13/13]
Processing E:/work/MS_project_2/data/new_pdfs\(정정공고문)25년1차청년매입임대_표준입주자모집공고문.pdf...
[                                        ] (0/1=[==                                      ] ( 1/14==[=====                       

### 임베딩 객체 생성

In [33]:
len(all_docs)

835

In [ ]:
import os
import uuid
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import AzureSearch

# ✅ 임베딩 객체 생성
embedding_api_key = ""
embedding_endpoint = ""
embedding_api_version = "2024-02-15-preview"
embedding_deployment = "text-embedding-3-small"

os.environ.pop("OPENAI_API_BASE", None)
os.environ.pop("BASE_URL", None)

embedding = AzureOpenAIEmbeddings(
    api_key=embedding_api_key,
    azure_endpoint=embedding_endpoint,
    model=embedding_deployment,
    openai_api_version=embedding_api_version
)

### 빈 인덱스 생성

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, SimpleField, SearchField, SearchFieldDataType,
    VectorSearch, HnswAlgorithmConfiguration, VectorSearchAlgorithmKind,
    VectorSearchProfile
)

ai_search_endpoint = ""
ai_search_api_key = ""
ai_search_index_name = "new_pdf_all_index"

index_client = SearchIndexClient(
    endpoint=ai_search_endpoint,
    credential=AzureKeyCredential(ai_search_api_key)
)

embedding_dim = 1536

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchField(name="content", type=SearchFieldDataType.String, searchable=True),
    SearchField(name="source", type=SearchFieldDataType.String, searchable=True, filterable=True),
    SearchField(
        name="embedding",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=embedding_dim,
        vector_search_profile_name="default"
    )
]

vector_search = VectorSearch(
    profiles=[VectorSearchProfile(name="default", algorithm_configuration_name="my-algorithm")],
    algorithms=[HnswAlgorithmConfiguration(name="my-algorithm", kind=VectorSearchAlgorithmKind.HNSW)]
)

index = SearchIndex(
    name=ai_search_index_name,
    fields=fields,
    vector_search=vector_search
)

# ✅ 인덱스 재생성
if ai_search_index_name in [i.name for i in index_client.list_indexes()]:
    index_client.delete_index(ai_search_index_name)
index_client.create_index(index)
print("✅ Azure Search 인덱스 생성 완료")

✅ Azure Search 인덱스 생성 완료


### 벡터스토어 객체 생성 및 임베딩

In [24]:
vectorstore = AzureSearch(
    azure_search_endpoint=ai_search_endpoint,
    azure_search_key=ai_search_api_key,
    index_name=ai_search_index_name,
    embedding_function=lambda x: x  # 더미 함수로 에러 방지
)


### 업로드

In [25]:
from langchain.schema import Document

# 1. 수동 임베딩 
texts = [doc.page_content for doc in all_docs]
metadatas = [{"source": doc.metadata.get("source", "")} for doc in all_docs]
ids = [str(uuid.uuid4()) for _ in all_docs]
embeddings = embedding.embed_documents(texts)

# 2. 문서 리스트 구성 (Document 객체)
documents = [Document(page_content=texts[i], metadata=metadatas[i]) for i in range(len(texts))]




RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2024-02-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}

In [15]:
print(texts[0],'\n\n')
print(metadatas[0],'\n\n')
print(ids[0],'\n\n')
print(embeddings[0],'\n\n')

###### 송파구, 용산구, 은평구, 종로구, 중구]


-----

###### ■ 공급대상 주택 : 총 190호

 • 주택군, 주택별 소재지, 면적, 임대조건 등 세부내역은 첨부 “주택내역” 참조

  * 위 공급호수는 기존에 선정된 예비입주자 계약 및 신규 매입물량 추가 등으로 변경될 수 있습니다.

  * 청약플러스(https://apply.lh.or.kr) 공고문 페이지에 주택사진이 게시되어 있으므로 참고하시기 바랍니다.

  * 아동복지시설(가정위탁 포함) 및 청소년쉼터 퇴소자 수시 우선공급으로 인해 지역별 공급물량이 

 감소될 수 있습니다.

  * 본 모집공고의 예비입주자로 선정되었다 하더라도 기존 임차인의 퇴거 및 임대공급 시행 전 실시하는

 주택 개보수 완료 상황 등에 따라 주택물량이 감소하거나 입주까지 상당기간이 소요될 수 있습니다.

 ■ 임대기간 및 임대조건

 구분 내용 


{'source': '서울지역본부 청년 매입임대주택.pdf'} 


4a854b5a-e3a5-4072-8746-fc86eaaa3bfa 


[-0.010760089382529259, 0.03576195240020752, 0.012796956114470959, 0.03143836185336113, 0.008831854909658432, -0.03041721321642399, -0.05253487080335617, -0.03369792550802231, -0.007397900801151991, -0.03526224195957184, 0.009396745823323727, -0.019456153735518456, 0.013220624066889286, -0.0060671474784612656, 0.014654578641057014, 0.002516209613531828, -0.05040566623210907, 0.016327526420354843, -0.02107478305697441, 0.023051902651786804, 0.033936917781829834

### 데이터 json형태로 변환

In [17]:
from azure.search.documents import SearchClient, IndexDocumentsBatch  # ✅ 여기!
from azure.core.credentials import AzureKeyCredential


# ✅ SearchClient 생성
search_client = SearchClient(
    endpoint=ai_search_endpoint,
    index_name=ai_search_index_name,
    credential=AzureKeyCredential(ai_search_api_key)
)

# ✅ 문서 생성 (Azure Search에 맞게 구조 변환)
docs_to_upload = []
for i in range(len(texts)):
    docs_to_upload.append({
        "id": ids[i],
        "content": texts[i],
        "source": metadatas[i]["source"],
        "embedding": embeddings[i],
    })


In [18]:
# ✅ 일괄 업로드
batch = IndexDocumentsBatch()
batch.add_upload_actions(docs_to_upload)

# ✅ 업로드 실행
search_client.index_documents(batch=batch)
print("✅ 벡터 업로드 완료")

✅ 벡터 업로드 완료
